In [1]:
import numpy as np
import pandas as pd
pd.pandas.set_option('display.max_columns',None)

In [2]:
def prepare_and_concat(index):
    
    # загрузка данных
    df_2017 = pd.read_excel(f'mixes_2017_{index}.xlsx')
    df_2018 = pd.read_excel(f'mixes_2018_{index}.xlsx')
    df_2019 = pd.read_excel(f'mixes_2019_{index}.xlsx')
    df_2020 = pd.read_excel(f'mixes_2020_{index}.xlsx')
    df_2021 = pd.read_excel(f'mixes_2021_{index}.xlsx')
    df_prop = pd.read_excel('properties.xlsx')
    
    # предобработка  df_prop
    # переимен столбцов для удобства работы
    df_prop.rename(columns={'Год' : 'year','Концентрат':'Conc','σ':'sigma','КТЦ эксп.':'KTC'},inplace=True)
    
    # удалене дубликатов
    df_prop=df_prop.drop_duplicates()
    
    
    # предобработка df_2019
    #  два столбца 31 и 31.1 объединим в один
    if 'Производитель 31.1' in df_2019.columns:
        df_2019[['Производитель 31','Производитель 31.1']] = df_2019[['Производитель 31','Производитель 31.1']].fillna(0)
        df_2019['Производитель 31'] += df_2019['Производитель 31.1']
        df_2019.drop(columns=['Производитель 31.1'],inplace=True)
    else:
        pass
    
    # предобработка df_2021
    # Удалим Столбец1
    if 'Столбец1' in df_2021.columns:
        df_2021 = df_2021.drop(columns=['Столбец1'])
    else:
        pass
    
    # собираем общий dataframe = df_mixes 
    df_mixes = pd.concat([df_2017,df_2018,df_2019,df_2020,df_2021])
    
    # предобработка df_mixes 
    # Объединим  Производитель 1 и Производитель 2 
    df_mixes[['Производитель 1 ','Производитель 2']] = df_mixes[['Производитель 1 ','Производитель 2']].fillna(0)
    df_mixes['Производитель 2'] += df_mixes['Производитель 1 ']
    df_mixes.drop(columns=['Производитель 1 '],inplace=True)
    
    #Добавим  столбец год, месяц,блок
    df_mixes['year']=df_mixes['Дата'].dt.year
    df_mixes['month']=df_mixes['Дата'].dt.month
    df_mixes['block'] = index
    
    #Избавимся от пропусков в дате(==2смена)
    df_mixes = df_mixes[~df_mixes['year'].isnull()]
    
    # замена тип данных
    df_mixes['year'] = df_mixes['year'].astype('int64')
    df_mixes['month'] = df_mixes['month'].astype('int64')
    
    #Удалим даты "13-08-2019", "14-08-2019", "17-11-2019"
    df_mixes = df_mixes.loc[~df_mixes['Дата'].isin(['2019-08-13','2019-08-14','2019-11-17'])].reset_index(drop=True)
    
    # объединение двух датафреймов df_mixes u df_prop
    def mixes_prop(df_mixes,df_prop):
    
        cols = ['R0', 'sigma', 'Vt', 'I', 'Io', 'SI', 'KTC', 'MF',
       'MF_spec', 'MF_otosh', 'CSR_carb',]
        idx = ['year', 'Conc']
    
        df_mixes = df_mixes.assign(group=lambda d: d.groupby('year').cumcount())
        a=df_prop.merge(df_mixes.melt(id_vars=['year', 'group'], var_name='Conc'), 
           on=idx).set_index(idx+['group']).reset_index().pivot_table(index=['year', 'group'], columns='Conc', fill_value=0)
    
        a.columns = a.columns.map('_'.join)
        data_mixes = (df_mixes.merge(a, left_on=['year', 'group'], right_index=True).drop(columns='group'))
        data_mixes = data_mixes.reindex(columns=sorted(data_mixes.columns.tolist()))
        # Заполним пропуски в столбцах Производительность {i}
        for i in range(1,60):
            if f'Производитель {i}' in data_mixes.columns:
                data_mixes[f'Производитель {i}'] = data_mixes[f'Производитель {i}'].fillna(0)
            else:
                continue
                
        return data_mixes
    
    data_mixes = mixes_prop(df_mixes,df_prop) # на выходе таблица датафреймов df_mixes + df_prop

    # определим средневзвешенное
    def mixes_prop_average(data_mixes):
    
        def added_col(X):
            xxx=0
            for i in range(1,60):
                column = '{}'.format(X)+' {}'.format(i)
                if column in data_mixes.columns:
                    xxx+= data_mixes[column]/100*data_mixes[f'Производитель {i}']
                    data_mixes.drop(columns=[column],inplace=True)
                else:
                    continue
                data_mixes[X+'_Aver']=xxx
    
            return data_mixes
        
        cols = ['R0_Производитель', 'sigma_Производитель', 'Vt_Производитель', 'I_Производитель', 'Io_Производитель', 
        'SI_Производитель', 'KTC_Производитель', 'MF_Производитель',
       'MF_spec_Производитель', 'MF_otosh_Производитель', 'CSR_carb_Производитель']

        for col in cols:
            added_col(col)
    
        return data_mixes

    data_mixes = mixes_prop_average(data_mixes) # на выходе df_mixes со средневзвешенными компонентами 
                                                # и удаленными столбцами из df_prop
            
    return data_mixes

In [3]:
def aligned_creats(dataframe):
    
    #формирование target
    def target_creat(dataframe):

        columns = ['Дата',
               'Качество кокса,% (Показатели прочности,% — CSR)',
               'Качество кокса,% (Показатели прочности,% — M10)',
               'Качество кокса,% (Показатели прочности,% — M25)']
        target = pd.DataFrame(dataframe, columns=columns)

        #rename columns
        target.rename(columns={'Дата':'ДатаПробыКокса', 
                            'Качество кокса,% (Показатели прочности,% — CSR)': 'CSR',
                            'Качество кокса,% (Показатели прочности,% — M10)':'M10',
                            'Качество кокса,% (Показатели прочности,% — M25)':'M25'}, inplace=True)
        # new column
        target["ДатаШихтовки"] = target["ДатаПробыКокса"] - pd.Timedelta(4,unit="d")
    
        return target
    
    target = target_creat(dataframe) # на выходе target
    
    #формирование признаков
    def feature_creat(dataframe):

        rename_cols = {
    "Дата": "ДатаШихтовки",
    "Состав шихты,%(спек.)": "% спек",
    "Состав шихты,%(в т.ч. Марка Ж)": "% марка Ж",
    "Состав шихты,%(в т.ч. кокс.)": "% кокс.",
    "Оборот печей, час": "Оборот печей, ч",
    "Качество шихты (помол,%)": "Помол, %",
    "Качество шихты (пыль,%)": "Пыль, %",
    "Качество шихты (Технический анализ, % — влага)": "Влага, %",
    "Качество шихты (Технический анализ, % — зола)": "Зола, %",
    "Качество шихты (Технический анализ, % — летуч.)": "Летуч., %",
    "Качество шихты (Мд. Серы,%)": "Мд. Серы, %",
    "Качество шихты (пласт. слой мм)": "Пласт. слой, мм"}
    
        add_cols=['R0_Производитель_Aver','sigma_Производитель_Aver',
          'Vt_Производитель_Aver','I_Производитель_Aver',
          'Io_Производитель_Aver', 'SI_Производитель_Aver','KTC_Производитель_Aver',
          'MF_Производитель_Aver','MF_spec_Производитель_Aver',
          'MF_otosh_Производитель_Aver','CSR_carb_Производитель_Aver','year','block','month']
    
        feature = pd.DataFrame(dataframe,columns=list(rename_cols.keys())+add_cols)
    
        #rename columns
        feature.rename(columns=rename_cols, inplace=True)
    
        return feature 
    
    feature = feature_creat(dataframe) # на выходе признаки
    
    #результирующая таблица
    def aligned_creat(feature,target):
        aligned_data = pd.merge(feature, target, on=["ДатаШихтовки"])
    
        #add columns
        aligned_data["Оборот печей отсеч, ч"] = np.where(aligned_data['Оборот печей, ч'] >= 18,
                                                        18, aligned_data['Оборот печей, ч'])
    
        return aligned_data

    aligned_data_all = aligned_creat(feature,target) # на выходе результирующая таблица
    
    return aligned_data_all

In [4]:
list_index = ['b1','b2']# перечисление блоков
dataframe = pd.DataFrame()# пустой датафрейм для 1-ой конкатенации
for index in range(len(list_index)):
    data_mixes = prepare_and_concat(list_index[index])
    aligned_creats_block = aligned_creats(data_mixes)
    dataframe = pd.concat([dataframe,aligned_creats_block])
dataframe = dataframe.reset_index(drop=True)    
dataframe.to_csv("aligned_data_b_all.csv")